In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
import random
warnings.filterwarnings('ignore')

print("📊 E-COMMERCE DATA ANALYTICS PROJECT")
print("="*60)
print("🎯 Business Intelligence & Customer Insights Analysis")

📊 E-COMMERCE DATA ANALYTICS PROJECT
🎯 Business Intelligence & Customer Insights Analysis


In [5]:
# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

In [6]:
# Generate customer data
n_customers = 5000
print(f"   📝 Generating {n_customers:,} customers...")

customers_data = {
    'customer_id': [f'CUST_{i:05d}' for i in range(1, n_customers + 1)],
    'age': np.random.normal(35, 12, n_customers).clip(18, 70).astype(int),
    'city': np.random.choice(['Mumbai', 'Delhi', 'Bangalore', 'Chennai', 'Hyderabad',
                             'Pune', 'Kolkata', 'Ahmedabad', 'Jaipur', 'Lucknow'], n_customers),
    'acquisition_channel': np.random.choice(['Organic Search', 'Social Media', 'Email',
                                           'Paid Ads', 'Referral'], n_customers,
                                          p=[0.3, 0.25, 0.2, 0.15, 0.1]),
    'registration_date': pd.date_range(start='2022-01-01', end='2024-12-31', periods=n_customers)
}

customers_df = pd.DataFrame(customers_data)

   📝 Generating 5,000 customers...


In [7]:
# Generate transaction data
n_transactions = 25000
print(f"   🛒 Generating {n_transactions:,} transactions...")

# Create more realistic transaction patterns
transaction_customers = np.random.choice(customers_df['customer_id'], n_transactions,
                                       p=np.random.dirichlet(np.ones(len(customers_df))))

transactions_data = {
    'transaction_id': [f'TXN_{i:06d}' for i in range(1, n_transactions + 1)],
    'customer_id': transaction_customers,
    'transaction_date': pd.date_range(start='2023-01-01', end='2024-12-31', periods=n_transactions) +
                       pd.to_timedelta(np.random.randint(0, 24*60, n_transactions), unit='m'),
    'product_category': np.random.choice(['Electronics', 'Clothing', 'Home & Garden', 'Books',
                                        'Sports', 'Beauty', 'Toys', 'Food & Beverages'], n_transactions,
                                       p=[0.2, 0.18, 0.15, 0.12, 0.1, 0.1, 0.08, 0.07]),
    'amount': np.random.lognormal(mean=6, sigma=1, size=n_transactions).clip(100, 50000),
    'payment_method': np.random.choice(['Credit Card', 'Debit Card', 'UPI', 'Net Banking', 'Wallet'],
                                     n_transactions, p=[0.3, 0.25, 0.2, 0.15, 0.1]),
    'discount_used': np.random.choice([0, 1], n_transactions, p=[0.7, 0.3]),
    'return_flag': np.random.choice([0, 1], n_transactions, p=[0.92, 0.08])
}

transactions_df = pd.DataFrame(transactions_data)
transactions_df['amount'] = transactions_df['amount'].round(2)

print(f"✅ Dataset generated successfully!")
print(f"   📊 {len(customers_df):,} customers with {len(transactions_df):,} transactions")

   🛒 Generating 25,000 transactions...
✅ Dataset generated successfully!
   📊 5,000 customers with 25,000 transactions


In [8]:
print("\n🔄 Step 1: Business Analytics Setup...")

# Set up analysis parameters
current_date = transactions_df['transaction_date'].max()
analysis_start_date = current_date - timedelta(days=365)  # Last 12 months

print(f"📅 Analysis Period: {analysis_start_date.date()} to {current_date.date()}")
print(f"📊 Dataset Overview: {len(customers_df):,} customers, {len(transactions_df):,} transactions")


🔄 Step 1: Business Analytics Setup...
📅 Analysis Period: 2024-01-01 to 2024-12-31
📊 Dataset Overview: 5,000 customers, 25,000 transactions


# STEP 2: CUSTOMER SEGMENTATION ANALYSIS (RFM)

In [9]:
print("\n🔄 Step 2: Customer Segmentation Analysis (RFM)...")

def perform_rfm_analysis(transactions_df, customers_df, current_date):
    """Perform RFM (Recency, Frequency, Monetary) Analysis"""

    print("   📈 Calculating RFM metrics...")

    # Calculate RFM for each customer
    rfm_data = transactions_df.groupby('customer_id').agg({
        'transaction_date': 'max',  # Last purchase date
        'transaction_id': 'count',  # Frequency
        'amount': 'sum'  # Monetary value
    }).reset_index()

    # Calculate Recency (days since last purchase)
    rfm_data['recency'] = (current_date - rfm_data['transaction_date']).dt.days
    rfm_data['frequency'] = rfm_data['transaction_id']
    rfm_data['monetary'] = rfm_data['amount']

    # Create RFM scores (1-5 scale)
    rfm_data['r_score'] = pd.qcut(rfm_data['recency'], 5, labels=[5,4,3,2,1])
    rfm_data['f_score'] = pd.qcut(rfm_data['frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])
    rfm_data['m_score'] = pd.qcut(rfm_data['monetary'], 5, labels=[1,2,3,4,5])

    # Combine RFM scores
    rfm_data['rfm_score'] = rfm_data['r_score'].astype(str) + rfm_data['f_score'].astype(str) + rfm_data['m_score'].astype(str)

    # Create customer segments based on RFM
    def segment_customers(row):
        if row['rfm_score'] in ['555', '554', '544', '545', '454', '455', '445']:
            return 'Champions'
        elif row['rfm_score'] in ['543', '444', '435', '355', '354', '345', '344', '335']:
            return 'Loyal Customers'
        elif row['rfm_score'] in ['512', '511', '422', '421', '412', '411', '311']:
            return 'New Customers'
        elif row['rfm_score'] in ['155', '154', '144', '214', '215', '115', '114']:
            return 'At Risk'
        elif row['rfm_score'] in ['155', '254', '245']:
            return 'Cannot Lose Them'
        elif row['rfm_score'] in ['331', '321', '231', '241', '251']:
            return 'Hibernating'
        else:
            return 'Others'

    rfm_data['customer_segment'] = rfm_data.apply(segment_customers, axis=1)

    return rfm_data[['customer_id', 'recency', 'frequency', 'monetary', 'customer_segment']]


🔄 Step 2: Customer Segmentation Analysis (RFM)...


In [10]:
# Perform RFM Analysis
rfm_results = perform_rfm_analysis(transactions_df, customers_df, current_date)

print(f"✅ RFM Analysis completed for {len(rfm_results):,} customers")

# Merge with customer data
customer_analytics = pd.merge(customers_df, rfm_results, on='customer_id', how='inner')

print("\n📊 Customer Segmentation Results:")
segment_summary = customer_analytics.groupby('customer_segment').agg({
    'customer_id': 'count',
    'monetary': ['mean', 'sum'],
    'frequency': 'mean',
    'recency': 'mean'
}).round(2)

segment_summary.columns = ['customer_count', 'avg_clv', 'total_revenue', 'avg_frequency', 'avg_recency']
segment_summary['revenue_percentage'] = (segment_summary['total_revenue'] / segment_summary['total_revenue'].sum() * 100).round(1)

for segment, data in segment_summary.iterrows():
    print(f"   • {segment}: {data['customer_count']:,} customers ({data['customer_count']/len(customer_analytics)*100:.1f}%)")
    print(f"     - Avg CLV: ₹{data['avg_clv']:,.2f}")
    print(f"     - Revenue Share: {data['revenue_percentage']:.1f}%")
    print(f"     - Avg Recency: {data['avg_recency']:.0f} days")

   📈 Calculating RFM metrics...
✅ RFM Analysis completed for 4,160 customers

📊 Customer Segmentation Results:
   • At Risk: 22.0 customers (0.5%)
     - Avg CLV: ₹5,068.66
     - Revenue Share: 0.7%
     - Avg Recency: 407 days
   • Cannot Lose Them: 39.0 customers (0.9%)
     - Avg CLV: ₹6,492.84
     - Revenue Share: 1.5%
     - Avg Recency: 193 days
   • Champions: 823.0 customers (19.8%)
     - Avg CLV: ₹9,685.67
     - Revenue Share: 47.6%
     - Avg Recency: 26 days
   • Hibernating: 64.0 customers (1.5%)
     - Avg CLV: ₹532.29
     - Revenue Share: 0.2%
     - Avg Recency: 117 days
   • Loyal Customers: 476.0 customers (11.4%)
     - Avg CLV: ₹6,253.12
     - Revenue Share: 17.8%
     - Avg Recency: 82 days
   • New Customers: 240.0 customers (5.8%)
     - Avg CLV: ₹689.36
     - Revenue Share: 1.0%
     - Avg Recency: 62 days
   • Others: 2,496.0 customers (60.0%)
     - Avg CLV: ₹2,098.17
     - Revenue Share: 31.3%
     - Avg Recency: 240 days


# STEP 3: REVENUE ANALYTICS & TRENDS

In [11]:
# Monthly revenue analysis
monthly_revenue = transactions_df.groupby(transactions_df['transaction_date'].dt.to_period('M')).agg({
    'amount': 'sum',
    'transaction_id': 'count',
    'customer_id': 'nunique'
}).reset_index()

monthly_revenue.columns = ['month', 'revenue', 'transactions', 'unique_customers']
monthly_revenue['avg_order_value'] = (monthly_revenue['revenue'] / monthly_revenue['transactions']).round(2)
monthly_revenue['revenue_per_customer'] = (monthly_revenue['revenue'] / monthly_revenue['unique_customers']).round(2)

print(f"📈 Monthly Revenue Trends (Last 12 months):")
for _, row in monthly_revenue.tail(12).iterrows():
    print(f"   • {row['month']}: ₹{row['revenue']:,.2f} | {row['transactions']:,} orders | AOV: ₹{row['avg_order_value']}")

# Calculate growth rates
monthly_revenue['revenue_growth'] = monthly_revenue['revenue'].pct_change() * 100
monthly_revenue['transaction_growth'] = monthly_revenue['transactions'].pct_change() * 100

latest_growth = monthly_revenue.iloc[-1]
print(f"\n📊 Latest Month Performance:")
print(f"   • Revenue Growth: {latest_growth['revenue_growth']:+.1f}%")
print(f"   • Transaction Growth: {latest_growth['transaction_growth']:+.1f}%")

📈 Monthly Revenue Trends (Last 12 months):
   • 2024-01: ₹709,365.64 | 1,061 orders | AOV: ₹668.58
   • 2024-02: ₹656,385.24 | 989 orders | AOV: ₹663.69
   • 2024-03: ₹715,619.42 | 1,061 orders | AOV: ₹674.48
   • 2024-04: ₹692,099.43 | 1,032 orders | AOV: ₹670.64
   • 2024-05: ₹726,101.71 | 1,060 orders | AOV: ₹685.0
   • 2024-06: ₹665,363.34 | 1,026 orders | AOV: ₹648.5
   • 2024-07: ₹690,239.87 | 1,063 orders | AOV: ₹649.33
   • 2024-08: ₹735,138.08 | 1,060 orders | AOV: ₹693.53
   • 2024-09: ₹675,905.75 | 1,028 orders | AOV: ₹657.5
   • 2024-10: ₹700,939.57 | 1,067 orders | AOV: ₹656.93
   • 2024-11: ₹691,003.29 | 1,020 orders | AOV: ₹677.45
   • 2024-12: ₹731,454.80 | 1,047 orders | AOV: ₹698.62

📊 Latest Month Performance:
   • Revenue Growth: +5.9%
   • Transaction Growth: +2.6%


# STEP 4: PRODUCT CATEGORY ANALYSIS

In [12]:
# Category performance analysis
category_analysis = transactions_df.groupby('product_category').agg({
    'amount': ['sum', 'mean', 'count'],
    'customer_id': 'nunique',
    'return_flag': 'sum',
    'discount_used': 'sum'
}).round(2)

category_analysis.columns = ['total_revenue', 'avg_order_value', 'total_orders', 'unique_customers', 'returns', 'discounts_used']
category_analysis['revenue_share'] = (category_analysis['total_revenue'] / category_analysis['total_revenue'].sum() * 100).round(1)
category_analysis['return_rate'] = (category_analysis['returns'] / category_analysis['total_orders'] * 100).round(1)
category_analysis['discount_rate'] = (category_analysis['discounts_used'] / category_analysis['total_orders'] * 100).round(1)

# Sort by revenue
category_analysis = category_analysis.sort_values('total_revenue', ascending=False)

print(f"🛍️ Product Category Performance:")
print(f"   Top 5 Categories by Revenue:")
for category, data in category_analysis.head(5).iterrows():
    print(f"   • {category}:")
    print(f"     - Revenue: ₹{data['total_revenue']:,.2f} ({data['revenue_share']:.1f}%)")
    print(f"     - Orders: {data['total_orders']:,} | AOV: ₹{data['avg_order_value']:,.2f}")
    print(f"     - Return Rate: {data['return_rate']:.1f}% | Discount Rate: {data['discount_rate']:.1f}%")

🛍️ Product Category Performance:
   Top 5 Categories by Revenue:
   • Electronics:
     - Revenue: ₹3,287,363.06 (19.6%)
     - Orders: 4,976.0 | AOV: ₹660.64
     - Return Rate: 8.0% | Discount Rate: 30.0%
   • Clothing:
     - Revenue: ₹2,976,613.02 (17.8%)
     - Orders: 4,489.0 | AOV: ₹663.09
     - Return Rate: 8.8% | Discount Rate: 30.7%
   • Home & Garden:
     - Revenue: ₹2,480,188.32 (14.8%)
     - Orders: 3,699.0 | AOV: ₹670.50
     - Return Rate: 7.7% | Discount Rate: 30.4%
   • Books:
     - Revenue: ₹2,047,607.06 (12.2%)
     - Orders: 2,991.0 | AOV: ₹684.59
     - Return Rate: 7.7% | Discount Rate: 30.5%
   • Beauty:
     - Revenue: ₹1,745,355.77 (10.4%)
     - Orders: 2,630.0 | AOV: ₹663.63
     - Return Rate: 7.8% | Discount Rate: 30.4%


# STEP 5: CUSTOMER BEHAVIOR ANALYTICS

In [13]:
# Customer behavior patterns
behavior_analysis = customer_analytics.groupby(['city', 'age']).agg({
    'monetary': 'mean',
    'frequency': 'mean',
    'customer_id': 'count'
}).reset_index()

behavior_analysis = behavior_analysis[behavior_analysis['customer_id'] >= 50]  # Filter for statistical significance

print(f"🌆 Geographic Performance (Top Cities by Avg CLV):")
city_performance = customer_analytics.groupby('city').agg({
    'monetary': 'mean',
    'frequency': 'mean',
    'customer_id': 'count'
}).sort_values('monetary', ascending=False).head(5)

for city, data in city_performance.iterrows():
    print(f"   • {city}: ₹{data['monetary']:,.2f} avg CLV | {data['frequency']:.1f} avg frequency | {data['customer_id']:,} customers")

# Age group analysis
customer_analytics['age_group'] = pd.cut(customer_analytics['age'],
                                        bins=[0, 25, 35, 45, 55, 100],
                                        labels=['18-25', '26-35', '36-45', '46-55', '55+'])

age_analysis = customer_analytics.groupby('age_group').agg({
    'monetary': 'mean',
    'frequency': 'mean',
    'customer_id': 'count'
}).round(2)

print(f"\n👥 Age Group Performance:")
for age_group, data in age_analysis.iterrows():
    print(f"   • {age_group}: ₹{data['monetary']:,.2f} avg CLV | {data['frequency']:.1f} avg frequency | {data['customer_id']:,} customers")

🌆 Geographic Performance (Top Cities by Avg CLV):
   • Jaipur: ₹4,427.68 avg CLV | 6.4 avg frequency | 418.0 customers
   • Delhi: ₹4,318.77 avg CLV | 6.5 avg frequency | 386.0 customers
   • Kolkata: ₹4,283.44 avg CLV | 6.2 avg frequency | 421.0 customers
   • Lucknow: ₹4,252.48 avg CLV | 6.3 avg frequency | 429.0 customers
   • Pune: ₹4,052.73 avg CLV | 6.3 avg frequency | 419.0 customers

👥 Age Group Performance:
   • 18-25: ₹3,807.39 avg CLV | 5.7 avg frequency | 943.0 customers
   • 26-35: ₹3,904.21 avg CLV | 5.9 avg frequency | 1,256.0 customers
   • 36-45: ₹4,160.54 avg CLV | 6.2 avg frequency | 1,229.0 customers
   • 46-55: ₹4,307.39 avg CLV | 6.4 avg frequency | 575.0 customers
   • 55+: ₹4,235.33 avg CLV | 6.0 avg frequency | 157.0 customers


# STEP 6: CHURN & RETENTION ANALYSIS

In [14]:
# Define churn based on recency
churn_threshold_days = 90
customer_analytics['is_churned'] = (customer_analytics['recency'] > churn_threshold_days).astype(int)
customer_analytics['customer_status'] = customer_analytics['is_churned'].map({0: 'Active', 1: 'Churned'})

churn_analysis = customer_analytics.groupby('customer_status').agg({
    'customer_id': 'count',
    'monetary': ['mean', 'sum']
}).round(2)

churn_analysis.columns = ['customer_count', 'avg_clv', 'total_value']

print(f"📉 Churn Analysis (>{churn_threshold_days} days inactive):")
for status, data in churn_analysis.iterrows():
    percentage = data['customer_count'] / len(customer_analytics) * 100
    print(f"   • {status}: {data['customer_count']:,} customers ({percentage:.1f}%)")
    print(f"     - Avg CLV: ₹{data['avg_clv']:,.2f}")
    print(f"     - Total Value: ₹{data['total_value']:,.2f}")

# High-risk customers (high value but churned)
high_value_threshold = customer_analytics['monetary'].quantile(0.8)
high_risk_customers = customer_analytics[
    (customer_analytics['is_churned'] == 1) &
    (customer_analytics['monetary'] > high_value_threshold)
]

print(f"\n🚨 High-Risk Analysis:")
print(f"   • {len(high_risk_customers):,} high-value customers have churned")
print(f"   • Revenue at risk: ₹{high_risk_customers['monetary'].sum():,.2f}")
if len(high_risk_customers) > 0:
    print(f"   • Avg value per high-risk customer: ₹{high_risk_customers['monetary'].mean():,.2f}")

📉 Churn Analysis (>90 days inactive):
   • Active: 1,900.0 customers (45.7%)
     - Avg CLV: ₹5,929.21
     - Total Value: ₹11,265,504.07
   • Churned: 2,260.0 customers (54.3%)
     - Avg CLV: ₹2,426.35
     - Total Value: ₹5,483,551.99

🚨 High-Risk Analysis:
   • 171 high-value customers have churned
   • Revenue at risk: ₹1,551,911.69
   • Avg value per high-risk customer: ₹9,075.51


# STEP 7: PAYMENT & CHANNEL ANALYTICS

In [15]:
# Payment method analysis
payment_analysis = transactions_df.groupby('payment_method').agg({
    'amount': ['sum', 'mean', 'count'],
    'return_flag': 'sum'
}).round(2)

payment_analysis.columns = ['total_revenue', 'avg_order_value', 'transaction_count', 'returns']
payment_analysis['return_rate'] = (payment_analysis['returns'] / payment_analysis['transaction_count'] * 100).round(1)
payment_analysis = payment_analysis.sort_values('total_revenue', ascending=False)

print(f"💳 Payment Method Performance:")
for method, data in payment_analysis.iterrows():
    print(f"   • {method}: ₹{data['total_revenue']:,.2f} | AOV: ₹{data['avg_order_value']:,.2f} | Return Rate: {data['return_rate']:.1f}%")

# Acquisition channel analysis
channel_performance = pd.merge(
    customers_df[['customer_id', 'acquisition_channel']],
    rfm_results[['customer_id', 'monetary', 'frequency']],
    on='customer_id'
)

channel_analysis = channel_performance.groupby('acquisition_channel').agg({
    'monetary': 'mean',
    'frequency': 'mean',
    'customer_id': 'count'
}).sort_values('monetary', ascending=False).round(2)

print(f"\n📈 Acquisition Channel Performance:")
for channel, data in channel_analysis.iterrows():
    print(f"   • {channel}: ₹{data['monetary']:,.2f} avg CLV | {data['frequency']:.1f} avg frequency | {data['customer_id']:,} customers")

💳 Payment Method Performance:
   • Credit Card: ₹5,060,986.92 | AOV: ₹679.42 | Return Rate: 7.9%
   • Debit Card: ₹4,194,436.75 | AOV: ₹668.01 | Return Rate: 8.0%
   • UPI: ₹3,191,523.46 | AOV: ₹641.00 | Return Rate: 8.4%
   • Net Banking: ₹2,575,555.52 | AOV: ₹682.27 | Return Rate: 8.5%
   • Wallet: ₹1,726,553.41 | AOV: ₹685.68 | Return Rate: 7.6%

📈 Acquisition Channel Performance:
   • Referral: ₹4,203.72 avg CLV | 6.3 avg frequency | 400.0 customers
   • Email: ₹4,069.41 avg CLV | 6.0 avg frequency | 844.0 customers
   • Paid Ads: ₹4,068.54 avg CLV | 6.0 avg frequency | 609.0 customers
   • Social Media: ₹3,975.85 avg CLV | 6.0 avg frequency | 1,057.0 customers
   • Organic Search: ₹3,962.22 avg CLV | 6.0 avg frequency | 1,250.0 customers


# STEP 8: BUSINESS RECOMMENDATIONS

In [16]:
# Top insights
total_customers = len(customer_analytics)
total_revenue = customer_analytics['monetary'].sum()
avg_clv = customer_analytics['monetary'].mean()
churn_rate = customer_analytics['is_churned'].mean() * 100

print(f"\n📊 Executive Summary:")
print(f"   • Total Customers: {total_customers:,}")
print(f"   • Total Revenue: ₹{total_revenue:,.2f}")
print(f"   • Average CLV: ₹{avg_clv:,.2f}")
print(f"   • Churn Rate: {churn_rate:.1f}%")

print(f"\n🎯 Strategic Recommendations:")

# Champions segment
champions = customer_analytics[customer_analytics['customer_segment'] == 'Champions']
print(f"   1. RETAIN CHAMPIONS ({len(champions):,} customers)")
print(f"      • Launch VIP loyalty program")
print(f"      • Offer exclusive early access to new products")
if len(champions) > 0:
    print(f"      • Average value: ₹{champions['monetary'].mean():,.2f}")

# At Risk segment
at_risk = customer_analytics[customer_analytics['customer_segment'] == 'At Risk']
if len(at_risk) > 0:
    print(f"   2. WIN BACK AT-RISK ({len(at_risk):,} customers)")
    print(f"      • Send personalized discount offers")
    print(f"      • Re-engagement email campaign")
    print(f"      • Potential revenue recovery: ₹{at_risk['monetary'].sum():,.2f}")

# High-value churned
if len(high_risk_customers) > 0:
    print(f"   3. RECOVER HIGH-VALUE CHURNED ({len(high_risk_customers):,} customers)")
    print(f"      • Immediate intervention required")
    print(f"      • Premium win-back offers")
    print(f"      • Revenue at stake: ₹{high_risk_customers['monetary'].sum():,.2f}")

# Category insights
top_category = category_analysis.index[0]
print(f"   4. OPTIMIZE {top_category.upper()} CATEGORY")
print(f"      • Top revenue generator: ₹{category_analysis.loc[top_category, 'total_revenue']:,.2f}")
print(f"      • Focus on inventory and marketing investment")


📊 Executive Summary:
   • Total Customers: 4,160
   • Total Revenue: ₹16,749,056.06
   • Average CLV: ₹4,026.22
   • Churn Rate: 54.3%

🎯 Strategic Recommendations:
   1. RETAIN CHAMPIONS (823 customers)
      • Launch VIP loyalty program
      • Offer exclusive early access to new products
      • Average value: ₹9,685.67
   2. WIN BACK AT-RISK (22 customers)
      • Send personalized discount offers
      • Re-engagement email campaign
      • Potential revenue recovery: ₹111,510.49
   3. RECOVER HIGH-VALUE CHURNED (171 customers)
      • Immediate intervention required
      • Premium win-back offers
      • Revenue at stake: ₹1,551,911.69
   4. OPTIMIZE ELECTRONICS CATEGORY
      • Top revenue generator: ₹3,287,363.06
      • Focus on inventory and marketing investment


# STEP 9: EXPORT ANALYTICS RESULTS

In [17]:
# Save comprehensive analytics results
analytics_summary = customer_analytics[['customer_id', 'age', 'city', 'acquisition_channel',
                                       'customer_segment', 'monetary', 'frequency', 'recency',
                                       'customer_status']].copy()

analytics_summary.to_csv('customer_analytics_results.csv', index=False)
print(f"✅ Customer analytics saved: customer_analytics_results.csv")

# Save segment summary
segment_summary.to_csv('customer_segment_summary.csv')
print(f"✅ Segment summary saved: customer_segment_summary.csv")

# Save category analysis
category_analysis.to_csv('product_category_analysis.csv')
print(f"✅ Category analysis saved: product_category_analysis.csv")

# Save high-risk customers
if len(high_risk_customers) > 0:
    high_risk_customers[['customer_id', 'monetary', 'recency', 'customer_segment']].to_csv('high_risk_customers_analytics.csv', index=False)
    print(f"✅ High-risk customers saved: high_risk_customers_analytics.csv")

# Save monthly trends
monthly_revenue.to_csv('monthly_revenue_trends.csv', index=False)
print(f"✅ Monthly trends saved: monthly_revenue_trends.csv")

print("\n✅ E-COMMERCE DATA ANALYTICS PROJECT COMPLETE!")
print("\n📋 FINAL ANALYTICS SUMMARY:")
print(f"   📊 Analyzed {total_customers:,} customers across {len(transactions_df):,} transactions")
print(f"   🎯 Identified {len(customer_analytics['customer_segment'].unique())} distinct customer segments")
print(f"   📈 Total business value analyzed: ₹{total_revenue:,.2f}")
print(f"   🚨 High-priority customers identified: {len(high_risk_customers):,}")
print(f"   💡 Generated {len(category_analysis)} category insights and strategic recommendations")

print("\n🎯 BUSINESS INTELLIGENCE READY FOR IMPLEMENTATION!")
print("📊 All analytics files exported for stakeholder review and action planning")

# Display sample results
print("\n📋 SAMPLE ANALYTICS PREVIEW:")
print("\nTop 10 Customer Analytics Results:")
print(analytics_summary.head(10).to_string(index=False))

print("\nCustomer Segment Distribution:")
print(customer_analytics['customer_segment'].value_counts().to_string())

print("\nTop Product Categories:")
print(category_analysis[['total_revenue', 'revenue_share']].head().to_string())

✅ Customer analytics saved: customer_analytics_results.csv
✅ Segment summary saved: customer_segment_summary.csv
✅ Category analysis saved: product_category_analysis.csv
✅ High-risk customers saved: high_risk_customers_analytics.csv
✅ Monthly trends saved: monthly_revenue_trends.csv

✅ E-COMMERCE DATA ANALYTICS PROJECT COMPLETE!

📋 FINAL ANALYTICS SUMMARY:
   📊 Analyzed 4,160 customers across 25,000 transactions
   🎯 Identified 7 distinct customer segments
   📈 Total business value analyzed: ₹16,749,056.06
   🚨 High-priority customers identified: 171
   💡 Generated 8 category insights and strategic recommendations

🎯 BUSINESS INTELLIGENCE READY FOR IMPLEMENTATION!
📊 All analytics files exported for stakeholder review and action planning

📋 SAMPLE ANALYTICS PREVIEW:

Top 10 Customer Analytics Results:
customer_id  age      city acquisition_channel customer_segment  monetary  frequency  recency customer_status
 CUST_00001   40 Ahmedabad        Social Media        Champions   9503.22     